In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [2]:
# 1. Cargar los datos
datos = pd.read_csv('water_potability.csv').dropna()
print("Forma por defecto del dataset '.csv': \n", list(datos.keys()))

# 2. Preprocesar los datos si es necesario
X = datos.drop('Potability', axis=1)
y = datos['Potability']

# 3. Dividir los datos en conjuntos de entrenamiento y prueba
features = list(datos.keys())
features.remove('Potability')
X_train3, X_temp3, y_train3, y_temp3 = train_test_split(datos[features], datos['Potability'], test_size=0.2)
X_val3, X_test3, y_val3, y_test3 = train_test_split(X_temp3, y_temp3, test_size=0.5)

Forma por defecto del dataset '.csv': 
 ['ph', 'Hardness', 'Solids', 'Chloramines', 'Sulfate', 'Conductivity', 'Organic_carbon', 'Trihalomethanes', 'Turbidity', 'Potability']


In [39]:
"""
Aquí secuestramos la función para normalizar que habíamos hecho en la tarea 1. Y luego la usamos para normalizar los datos de la tarea 3, en el punto 3.
Los datos de validación y test se normalizaron con la media y la desviación de los datos de entrenamiento
"""
parametros_train = dict(map(lambda x: (x, {'media': np.mean(X_train3[x]), 'desviacion': np.std(X_train3[x])}), X_train3.keys()))
def normalizar(dataSet, features, parametros = []):
    """
    Esta función resta la media y luego divide entre la desviación estandar de cada dato en cada columna. Si se le especifica la media y la desviación en los parámetros 
    Usa esa media y desviación dada, de lo contrario, calcula la media y la desviación de cada columna especificada en "features"
    @input: dataSet <pd.DataFrame>: Tabla de datos a la que se le va a dar normalización a la columans dadas
    @input: features <list>: lista de "features" o columnas que se van a normalizar
    @input: (opcional) parametros <dict>: En caso de que se quiera normalizar con una media y una desviación dada se especifican en un diccionario así: {'mileage':{'media': 2.9, 'desviacion': 7}}
    """
    if len(parametros) == 0:
        for feature in features:
            media = np.mean(dataSet[feature])
            desviacion = np.std(dataSet[feature])
            if desviacion != 0:
                dataSet[feature] = (dataSet[feature] - media)/desviacion
            #fin if 
        #fin for
    #fin if 
    else:
        for feature in features:
            media = parametros[feature]['media']
            desviacion = parametros[feature]['desviacion']
            if desviacion != 0:
                dataSet[feature] = (dataSet[feature] - media)/desviacion
            #fin if 
        #fin for
#fin función
normalizar(X_train3, features)
normalizar(X_val3, features, parametros_train)
normalizar(X_test3, features, parametros_train)
X_train3['D'] = (1/len(X_train3.index))*np.ones(len(X_train3.index))

In [72]:
def funcion_error(limite, feature, no_potables, potables):
    no_potables_izq = no_potables[no_potables[feature] <= limite]
    potables_izq = potables[potables[feature] <= limite]
    no_potables_der = no_potables[no_potables[feature] > limite]
    potables_der = potables[potables[feature] > limite]
    D_tot_izq = sum(no_potables_izq['D']) + sum(potables_izq['D'])
    D_tot_der = sum(no_potables_der['D']) + sum(potables_der['D'])
    if D_tot_der == 0 or D_tot_izq == 0:
        return 1
    #fin if 
    x_0_izq = sum(no_potables_izq['D'])/D_tot_izq
    x_1_izq = sum(potables_izq['D'])/D_tot_izq
    x_0_der = sum(no_potables_der['D'])/D_tot_der
    x_1_der = sum(potables_der['D'])/D_tot_der
    I_izq = 1 - x_0_izq**2 - x_1_izq**2
    I_der = 1 - x_0_der**2 - x_1_der**2
    Impureza_total = I_izq*D_tot_izq + I_der*D_tot_der
    return Impureza_total
#fin función

    
def MinimizarImpureza_feature(datos, etiquetas, feature, debug = 'off'):
    potables = datos[etiquetas == 1]
    no_potables = datos[etiquetas == 0]
    g_0 = no_potables['D'].dot(no_potables[feature])
    g_1 = potables['D'].dot(potables[feature])
    if g_0 > g_1:
        g_mayor = g_0
        g_menor = g_1
        etiqueta_menor = 1
    else:
        g_mayor = g_1
        g_menor = g_0
        etiqueta_menor = 0
    #fin if 
    if debug == 'on':
        datos_busqueda = datos
    else:
        datos_busqueda = datos[(datos[feature] > g_menor) & (datos[feature] < g_mayor)]
    #fin if 
    g_medio = sum(no_potables['D'])*g_0 + sum(potables['D'])*g_1
    if  len(datos_busqueda) == 0:
        return {'valor_limite': g_medio, 'etiqueta_menor': etiqueta_menor}
    else:
        limite_optimo = g_medio
        impureza_optimo = funcion_error(limite_optimo, feature, no_potables, potables)
        for i in datos_busqueda.index:
            impureza_evaluar = funcion_error(datos_busqueda[feature][i], feature, no_potables, potables)
            if  impureza_evaluar < impureza_optimo:
                limite_optimo = datos_busqueda[feature][i]
                impureza_optimo = impureza_evaluar
            #fin if 
        #fin for 
        return {'valor_limite': limite_optimo, 'etiqueta_menor': etiqueta_menor}
    #fin if
#fin función

In [73]:
print(MinimizarImpureza_feature(X_train3, y_train3, 'Solids'))

{'valor_limite': 3.2870716519534713, 'etiqueta_menor': 0, 'g_medio': -0.004207831814974769, 'p_opt': 0.48036624855771604}


In [11]:
lista = [True, True, False]
print(lista and lista)

[True, True, False]
